
# Implementation of coordinate descent optimization algorithm

# Solving for artificial Data

In [295]:
import pandas as pd
import numpy as np
import bz2
import time

In [298]:
(np.random.normal(1, 1, f_d)*np.random.choice([0, 1], size=f_d, p=[.9, .1])).shape
(np.random.normal(1, 1, f_d)*np.random.choice([0, 1], size=f_d, p=[.9, .1]))[51,] = 1


IndexError: index 51 is out of bounds for axis 0 with size 50

In [301]:
# New data test
f_d = 50
d_d = 5000
X = np.ones((d_d,f_d))
Y = np.zeros(d_d)
for i in range(d_d):
  if i < d_d/2:
    X[i,:f_d-1] = np.random.normal(1, 1, f_d-1)*np.random.choice([0, 1], size=f_d-1, p=[.9, .1])
    X[i, f_d-1] = 1
    
    Y[i] = 1
  else:
    X[i,:f_d-1] = np.random.normal(-1, 1, f_d-1)*np.random.choice([0, 1], size=f_d-1, p=[.9, .1])
    X[i, f_d-1] = 1
    Y[i] = -1


In [23]:
# bias term
X[:, f_d-1]


array([1., 1., 1., ..., 1., 1., 1.])

In [300]:
# maybe not ? ;) - not run for now
for i in range(f_d):
  mean = X[:,i].mean()
  std = X[:,i].std()
  if std == 0: #If column of zeros
    std = 1
    print("Std is Zero")
  X[:,i] = (X[:,i] - mean)/std

Std is Zero


In [302]:
X

array([[ 0.        ,  0.        ,  0.46250836, ...,  0.        ,
         0.        ,  1.        ],
       [-0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.43991082,  1.        ],
       [ 0.        , -0.        ,  0.        , ...,  0.        ,
        -0.        ,  1.        ],
       ...,
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        ,  1.        ],
       [-0.        , -0.        , -0.        , ...,  0.        ,
        -0.        ,  1.        ],
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        ,  1.        ]])

In [310]:
# algorithm described in https://jmlr.org/papers/volume9/chang08a/chang08a.pdf


n_of_features = f_d
n_of_observations =  d_d


# initial weights

#weights = np.ones(shape = (1,n_of_features) ) * 0.01 # initial weights of predictors
weights = np.ones((n_of_features)) * 0.01

# hyperparameters
beta = 0.5
sigma = 0.01
C = 5

# outer iterations:
for k in range(50):
    
    print("outer iteration - ", k)
    acc = 0.0
    for i in range(d_d):
      if np.dot(weights,X[i,:])*Y[i] < 0:
        acc += 1
    print("Accuracy",acc/d_d)
    print("Weights sum:",weights.sum())
    print("\n")
    
    
    # inner iterations shuld be as many as the number of features
    for i in range(n_of_features):
        if(i%100==0):
          print("feature " , i)
        shuffle = np.random.choice(n_of_observations, n_of_observations, replace = False) # shuffling recommended by authors
        col = X[:,i] #index2Col(X, i)
        
        # reorder randomly column of X and target (Y):
        col_ = col[shuffle]
        Y_ = Y[shuffle] #np.array(Y.iloc[shuffle])
        X_ = X[shuffle,:]
        
        # solving the sub-problem and update weight_i
        
        ### calculating Newton direction d
        
        D_prim_0 = 0
        
        sum_of_squares_of_column = 0
        
        #----- calculating D'  for z = 0 , in the article: equation (9)
        for j in range(n_of_observations): 
            #----- b_j defined in thr article, part of D' calculations
            b_j = max(Y_[j]*float(np.dot(weights,X_[j,:])),0)
            #print(float(np.dot(weights,index2Row(df,j))),b_j)
            if b_j > 0.0: #----- indicator defined in the article
                D_prim_0 += Y_[j]*col_[j]*b_j
                sum_of_squares_of_column += col_[j]**2
            
            
        D_prim_0 *= (-2*C)

        #D_prim_0 += weights[0,i]
        #D_prim_0 += weights[i]


        #----- calculating D''  for z = 0 , in the article: equation (10)
        D_bis_0 = 1 + 2*C*sum_of_squares_of_column

        #-----  equation (13) in the article
        d = -D_prim_0/D_bis_0 
        ### compute lambda = max{1, beta, beta^2, beta^3, ...}
        
        # we do this to ensure sufficient convergence condition & to make calculations quicker
        # Theorem 1 ( from article) indicated that lambda_ (14) fulfilling (15) 
        
        H_i = 1 + 2*C*np.dot(col_.T,col_)
        lambda_ = D_bis_0 /((H_i/2)+sigma)
        
        lambd = 1
        while lambd > lambda_:
            lambd *= beta
            
        # update weights
        weights[i] -= lambd*d






outer iteration -  0
Accuracy 0.0474
Weights sum: 0.5


feature  0
outer iteration -  1
Accuracy 0.8334
Weights sum: -0.3229048132794372


feature  0
outer iteration -  2
Accuracy 0.8798
Weights sum: -0.35671587809615235


feature  0
outer iteration -  3
Accuracy 0.901
Weights sum: -0.37500824237923064


feature  0
outer iteration -  4
Accuracy 0.9114
Weights sum: -0.38341040904981033


feature  0
outer iteration -  5
Accuracy 0.9196
Weights sum: -0.3895818341274378


feature  0
outer iteration -  6
Accuracy 0.9264
Weights sum: -0.39426929968320484


feature  0
outer iteration -  7
Accuracy 0.9316
Weights sum: -0.397776057926229


feature  0
outer iteration -  8
Accuracy 0.939
Weights sum: -0.4003992587160813


feature  0
outer iteration -  9
Accuracy 0.9422
Weights sum: -0.402040683879125


feature  0
outer iteration -  10
Accuracy 0.9438
Weights sum: -0.40321297120464716


feature  0
outer iteration -  11
Accuracy 0.9462
Weights sum: -0.40450544141615125


feature  0
outer iteration 

KeyboardInterrupt: 

In [304]:
from sklearn.svm import LinearSVC


svc = LinearSVC(penalty = "l1", loss = "squared_hinge", max_iter = 100, C = 5, dual = False) # last 2 params - same as in our algorithm


In [315]:
svc.fit(X,Y)

svc.score(X,Y)

0.9738

In [ ]:
# below -  no used

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, Y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [14]:
clf.score(X,Y)

0.9778

# 0. Smoking data

In [70]:
!ls data

real-sim.bz2
real-sim-col.csv
smoking.csv
X.csv
Y.csv


In [272]:
 # https://www.kaggle.com/datasets/kukuroo3/body-signal-of-smoking

data = pd.read_csv('data/smoking.csv')

In [273]:
Y = data['smoking']
Y[Y == 0.0] = -1

C:\Users\Patryk\AppData\Local\Temp/ipykernel_12172/223503117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[Y == 0.0] = -1


In [274]:
data = data.iloc[0:-2,0:-1]

In [275]:
np.unique(data['oral'])

array(['Y'], dtype=object)

In [276]:
data.loc[data['gender'] == 'M', 'gender'] = 1
data.loc[data['gender'] == 'F', 'gender'] = 0

data.loc[data['tartar'] == 'Y', 'tartar'] = 1
data.loc[data['tartar'] == 'N', 'tartar'] = 0

In [277]:
data = data.drop('oral', axis=1)

In [278]:
data

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,tartar
0,0,0,40,155,60,81.3,1.2,1.0,1.0,1.0,...,73.0,126.0,12.9,1.0,0.7,18.0,19.0,27.0,0,1
1,1,0,40,160,60,81.0,0.8,0.6,1.0,1.0,...,42.0,127.0,12.7,1.0,0.6,22.0,19.0,18.0,0,1
2,2,1,55,170,60,80.0,0.8,0.8,1.0,1.0,...,55.0,151.0,15.8,1.0,1.0,21.0,16.0,22.0,0,0
3,3,1,40,165,70,88.0,1.5,1.5,1.0,1.0,...,45.0,226.0,14.7,1.0,1.0,19.0,26.0,18.0,0,1
4,4,0,40,155,60,86.0,1.0,1.0,1.0,1.0,...,62.0,107.0,12.5,1.0,0.6,16.0,14.0,22.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55685,55671,1,40,170,80,90.5,1.2,1.5,1.0,1.0,...,57.0,135.0,14.8,1.0,0.9,16.0,28.0,68.0,0,1
55686,55673,0,60,150,50,75.0,1.0,1.2,1.0,1.0,...,52.0,116.0,12.6,1.0,0.8,26.0,21.0,14.0,0,1
55687,55676,0,40,170,65,75.0,0.9,0.9,1.0,1.0,...,75.0,118.0,12.3,1.0,0.6,14.0,7.0,10.0,1,1
55688,55681,0,45,160,50,70.0,1.2,1.2,1.0,1.0,...,73.0,79.0,14.0,1.0,0.9,20.0,12.0,14.0,0,1


In [279]:
Y = Y[0:-2]

In [258]:
Y

0       -1
1       -1
2        1
3       -1
4       -1
        ..
55685   -1
55686   -1
55687   -1
55688   -1
55689   -1
Name: smoking, Length: 55690, dtype: int64

In [99]:
data.iloc[:,1:25]

,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,tartar
0,0,40,155,60,81.3,1.2,1.0,1.0,1.0,114.0,...,73.0,126.0,12.9,1.0,0.7,18.0,19.0,27.0,0,1
1,0,40,160,60,81.0,0.8,0.6,1.0,1.0,119.0,...,42.0,127.0,12.7,1.0,0.6,22.0,19.0,18.0,0,1
2,1,55,170,60,80.0,0.8,0.8,1.0,1.0,138.0,...,55.0,151.0,15.8,1.0,1.0,21.0,16.0,22.0,0,0
3,1,40,165,70,88.0,1.5,1.5,1.0,1.0,100.0,...,45.0,226.0,14.7,1.0,1.0,19.0,26.0,18.0,0,1
4,0,40,155,60,86.0,1.0,1.0,1.0,1.0,120.0,...,62.0,107.0,12.5,1.0,0.6,16.0,14.0,22.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55685,1,40,170,80,90.5,1.2,1.5,1.0,1.0,130.0,...,57.0,135.0,14.8,1.0,0.9,16.0,28.0,68.0,0,1
55686,0,60,150,50,75.0,1.0,1.2,1.0,1.0,102.0,...,52.0,116.0,12.6,1.0,0.8,26.0,21.0,14.0,0,1
55687,0,40,170,65,75.0,0.9,0.9,1.0,1.0,110.0,...,75.0,118.0,12.3,1.0,0.6,14.0,7.0,10.0,1,1
55688,0,45,160,50,70.0,1.2,1.2,1.0,1.0,101.0,...,73.0,79.0,14.0,1.0,0.9,20.0,12.0,14.0,0,1


In [280]:
X = np.asarray(data.iloc[:,1:25])

In [281]:
X.shape

(55690, 24)

In [125]:
# X = np.concatenate([X, np.ones(shape = (X.shape[0], 1))], axis=1)

In [282]:
Y

0       -1
1       -1
2        1
3       -1
4       -1
        ..
55685   -1
55686   -1
55687   -1
55688   -1
55689   -1
Name: smoking, Length: 55690, dtype: int64

In [283]:
for i in range(24):
  print(i)
  meann = X[:,i].mean()
  std = X[:,i].std()
  if std == 0: #If column of zeros
    std = 1
    print("Std is Zero")
    print(i)
  X[:,i] = (X[:,i] - meann)/std

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [284]:
X = np.concatenate([X, np.ones(shape = (X.shape[0], 1))], axis=1)

In [285]:
X.shape

(55690, 25)

In [286]:
Y.shape

(55690,)

In [287]:
np.unique(Y, return_counts = True)

(array([-1,  1], dtype=int64), array([35236, 20454], dtype=int64))

In [ ]:
# Y = Y[0:-2]

In [288]:
svc.fit(X,Y)

svc.score(X,Y)

0.7457532770694918

In [289]:
pd.DataFrame(X).describe()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
count,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,...,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.000000,55690.0
unique,2.000000,14.000000,13.000000,22.000000,566.000000,19.000000,17.000000,2.000000,2.000000,130.000000,...,289.000000,145.000000,6.000000,38.000000,219.000000,245.000000,488.000000,2.000000,2.000000,1.0
top,0.757105,-0.346479,0.581925,-0.067475,-0.220657,0.384843,0.396224,-0.162049,-0.163849,-0.840447,...,-0.121274,0.496903,-0.215406,0.064361,-0.319445,-0.388914,-0.496165,-0.520769,0.894424,1.0
freq,35399.000000,15181.000000,11381.000000,8195.000000,1917.000000,12746.000000,12539.000000,54265.000000,54234.000000,3490.000000,...,727.000000,1522.000000,52597.000000,11293.000000,3816.000000,2744.000000,2117.000000,43809.000000,30939.000000,55690.0


In [291]:
# algorithm described in https://jmlr.org/papers/volume9/chang08a/chang08a.pdf


n_of_features = X.shape[1]
n_of_observations = Y.shape[0]


# initial weights

#weights = np.ones(shape = (1,n_of_features) ) * 1 # initial weights of predictors
weights = np.ones((n_of_features)) * 100

# hyperparameters
beta = 0.5
sigma = 0.01
C = 5

# outer iterations:
for k in range(100):
    # inner iterations shuld be as many as the number of features
    for i in range(n_of_features):
#         if(i%100==0):
#           print("feature " , i)
        shuffle = np.random.choice(n_of_observations, n_of_observations, replace = False) # shuffling recommended by authors
        col = X[:,i] #index2Col(X, i)
        
        # reorder randomly column of X and target (Y):
        col_ = col[shuffle]
        Y_ = Y[shuffle] #np.array(Y.iloc[shuffle])
        X_ = X[shuffle,:]
        
        # solving the sub-problem and update weight_i
        
        ### calculating Newton direction d
        
        D_prim_0 = 0
        
        sum_of_squares_of_column = 0
        
        #----- calculating D'  for z = 0 , in the article: equation (9)
        for j in range(n_of_observations): 
            #----- b_j defined in thr article, part of D' calculations
            b_j = max(Y_[j]*float(np.dot(weights,X_[j,:])),0)
            #print(float(np.dot(weights,index2Row(df,j))),b_j)
            if b_j > 0.0: #----- indicator defined in the article
                D_prim_0 += Y_[j]*col_[j]*b_j
                sum_of_squares_of_column += col_[j]**2
            
            
        D_prim_0 *= (-2*C)

        #D_prim_0 += weights[0,i]
        #D_prim_0 += weights[i]


        #----- calculating D''  for z = 0 , in the article: equation (10)
        D_bis_0 = 1 + 2*C*sum_of_squares_of_column

        #-----  equation (13) in the article
        d = -D_prim_0/D_bis_0 
        ### compute lambda = max{1, beta, beta^2, beta^3, ...}
        
        # we do this to ensure sufficient convergence condition & to make calculations quicker
        # Theorem 1 ( from article) indicated that lambda_ (14) fulfilling (15) 
        
        H_i = 1 + 2*C*np.dot(col_.T,col_)
        lambda_ = D_bis_0 /((H_i/2)+sigma)
        
        lambd = 1
        while lambd > lambda_:
            lambd *= beta
            
        # update weights
        weights[i] -= lambd*d
    print("outer iteration - ", k)
    acc = 0.0
    for i in range(n_of_observations):
      if np.dot(weights,X[i,:])*Y[i] < 0:
        acc += 1
    print("Accuracy ",acc/n_of_observations)
    print("Weights sum: ",weights.sum())
    print("\n")



outer iteration -  0
Accuracy  0.6253007721314419
Weights sum:  498.95378341584404


outer iteration -  1
Accuracy  0.6090321422158377
Weights sum:  275.1969941745641


outer iteration -  2
Accuracy  0.6092835338480876
Weights sum:  162.31202084012085


outer iteration -  3
Accuracy  0.6088705333093912
Weights sum:  101.40987879415431


outer iteration -  4
Accuracy  0.6003411743580536
Weights sum:  91.45643698678285


outer iteration -  5
Accuracy  0.5992817381935716
Weights sum:  81.78674513724415


outer iteration -  6
Accuracy  0.5986891722032681
Weights sum:  67.6501794594635


outer iteration -  7
Accuracy  0.5949542108098402
Weights sum:  62.417138611079366


outer iteration -  8
Accuracy  0.5932662955647333
Weights sum:  55.696527748840765




KeyboardInterrupt: 

In [294]:
# algorithm described in https://jmlr.org/papers/volume9/chang08a/chang08a.pdf

X = X[0:10000,:]
Y = Y[0:10000]

n_of_features = X.shape[1]
n_of_observations = Y.shape[0]


# initial weights

#weights = np.ones(shape = (1,n_of_features) ) * 1 # initial weights of predictors
weights = np.ones((n_of_features)) * -1000

# hyperparameters
beta = 0.5
sigma = 0.01
C = 5

# outer iterations:
for k in range(100):
    # inner iterations shuld be as many as the number of features
    for i in range(n_of_features):
#         if(i%100==0):
#           print("feature " , i)
        shuffle = np.random.choice(n_of_observations, n_of_observations, replace = False) # shuffling recommended by authors
        col = X[:,i] #index2Col(X, i)
        
        # reorder randomly column of X and target (Y):
        col_ = col[shuffle]
        Y_ = Y[shuffle] #np.array(Y.iloc[shuffle])
        X_ = X[shuffle,:]
        
        # solving the sub-problem and update weight_i
        
        ### calculating Newton direction d
        
        D_prim_0 = 0
        
        sum_of_squares_of_column = 0
        
        #----- calculating D'  for z = 0 , in the article: equation (9)
        for j in range(n_of_observations): 
            #----- b_j defined in thr article, part of D' calculations
            b_j = max(Y_[j]*float(np.dot(weights,X_[j,:])),0)
            #print(float(np.dot(weights,index2Row(df,j))),b_j)
            if b_j > 0.0: #----- indicator defined in the article
                D_prim_0 += Y_[j]*col_[j]*b_j
                sum_of_squares_of_column += col_[j]**2
            
            
        D_prim_0 *= (-2*C)

        #D_prim_0 += weights[0,i]
        #D_prim_0 += weights[i]


        #----- calculating D''  for z = 0 , in the article: equation (10)
        D_bis_0 = 1 + 2*C*sum_of_squares_of_column

        #-----  equation (13) in the article
        d = -D_prim_0/D_bis_0 
        ### compute lambda = max{1, beta, beta^2, beta^3, ...}
        
        # we do this to ensure sufficient convergence condition & to make calculations quicker
        # Theorem 1 ( from article) indicated that lambda_ (14) fulfilling (15) 
        
        H_i = 1 + 2*C*np.dot(col_.T,col_)
        lambda_ = D_bis_0 /((H_i/2)+sigma)
        
        lambd = 1
        while lambd > lambda_:
            lambd *= beta
            
        # update weights
        weights[i] -= lambd*d*0.4
    print("outer iteration - ", k)
    acc = 0.0
    for i in range(n_of_observations):
      if np.dot(weights,X[i,:])*Y[i] < 0:
        acc += 1
    print("Accuracy ",acc/n_of_observations)
    print("Weights sum: ",weights.sum())
    print("\n")



outer iteration -  0
Accuracy  0.5522
Weights sum:  -9199.073407535197


outer iteration -  1
Accuracy  0.4951
Weights sum:  -4475.236608463071


outer iteration -  2
Accuracy  0.5087
Weights sum:  -2804.952660218844


outer iteration -  3
Accuracy  0.5247
Weights sum:  -1661.8033055324133


outer iteration -  4
Accuracy  0.5302
Weights sum:  -1088.5515212544576


outer iteration -  5
Accuracy  0.5342
Weights sum:  -761.6372767355559


outer iteration -  6
Accuracy  0.5428
Weights sum:  -605.8393753368822


outer iteration -  7
Accuracy  0.5454
Weights sum:  -511.4572129320163


outer iteration -  8
Accuracy  0.5471
Weights sum:  -400.10844535950207


outer iteration -  9
Accuracy  0.5472
Weights sum:  -341.9819554342586


outer iteration -  10
Accuracy  0.5471
Weights sum:  -312.27766798518


outer iteration -  11
Accuracy  0.5508
Weights sum:  -302.4231996715514


outer iteration -  12
Accuracy  0.5519
Weights sum:  -250.058006648105


outer iteration -  13
Accuracy  0.5512
Weights s

KeyboardInterrupt: 

# 1. Banknote authentication data

In [163]:
!ls data

data_banknote_authentication.txt
real-sim.bz2
real-sim-col.csv
smoking.csv
X.csv
Y.csv


In [167]:
# https://archive.ics.uci.edu/ml/datasets/banknote+authentication

data = pd.read_csv("data/data_banknote_authentication.txt", header = None)

In [198]:
X = data.iloc[:,0:4]
Y = data.iloc[:,4]

label_mapping = {0: -1,
                1: 1}

Y = Y.map(label_mapping)

X = np.array(X)
Y = np.array(Y)

# free term
X = np.concatenate([X, np.ones(shape = (X.shape[0], 1))], axis=1)

In [199]:
# svc from library

svc.fit(X,Y)

svc.score(X,Y)



C:\Users\Patryk\anaconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9905247813411079

In [202]:

# algorithm described in https://jmlr.org/papers/volume9/chang08a/chang08a.pdf


n_of_features = X.shape[1]
n_of_observations = Y.shape[0]


# initial weights

#weights = np.ones(shape = (1,n_of_features) ) * 1 # initial weights of predictors
weights = np.ones((n_of_features)) * -100

# hyperparameters
beta = 0.5
sigma = 0.01
C = 5

# outer iterations:
for k in range(1000):
    # inner iterations shuld be as many as the number of features
    for i in range(n_of_features):
#         if(i%100==0):
#           print("feature " , i)
        shuffle = np.random.choice(n_of_observations, n_of_observations, replace = False) # shuffling recommended by authors
        col = X[:,i] #index2Col(X, i)
        
        # reorder randomly column of X and target (Y):
        col_ = col[shuffle]
        Y_ = Y[shuffle] #np.array(Y.iloc[shuffle])
        X_ = X[shuffle,:]
        
        # solving the sub-problem and update weight_i
        
        ### calculating Newton direction d
        
        D_prim_0 = 0
        
        sum_of_squares_of_column = 0
        
        #----- calculating D'  for z = 0 , in the article: equation (9)
        for j in range(n_of_observations): 
            #----- b_j defined in thr article, part of D' calculations
            b_j = max(Y_[j]*float(np.dot(weights,X_[j,:])),0)
            #print(float(np.dot(weights,index2Row(df,j))),b_j)
            if b_j > 0.0: #----- indicator defined in the article
                D_prim_0 += Y_[j]*col_[j]*b_j
                sum_of_squares_of_column += col_[j]**2
            
            
        D_prim_0 *= (-2*C)

        #D_prim_0 += weights[0,i]
        #D_prim_0 += weights[i]


        #----- calculating D''  for z = 0 , in the article: equation (10)
        D_bis_0 = 1 + 2*C*sum_of_squares_of_column

        #-----  equation (13) in the article
        d = -D_prim_0/D_bis_0 
        ### compute lambda = max{1, beta, beta^2, beta^3, ...}
        
        # we do this to ensure sufficient convergence condition & to make calculations quicker
        # Theorem 1 ( from article) indicated that lambda_ (14) fulfilling (15) 
        
        H_i = 1 + 2*C*np.dot(col_.T,col_)
        lambda_ = D_bis_0 /((H_i/2)+sigma)
        
        lambd = 1
        while lambd > lambda_:
            lambd *= beta
            
        # update weights
        weights[i] -= lambd*d
    if k % 100 == 0:
        print("outer iteration - ", k)
        acc = 0.0
        for i in range(n_of_observations):
          if np.dot(weights,X[i,:])*Y[i] < 0:
            acc += 1
        print("Accuracy ",acc/n_of_observations)
        print("Weights sum: ",weights.sum())
        print("\n")



outer iteration -  0
Accuracy  0.9876093294460642
Weights sum:  17.076346184748036


outer iteration -  100
Accuracy  0.9876093294460642
Weights sum:  17.79406466593265


outer iteration -  200
Accuracy  0.9876093294460642
Weights sum:  18.472612473613285


outer iteration -  300
Accuracy  0.9876093294460642
Weights sum:  19.112678979842364


outer iteration -  400
Accuracy  0.9912536443148688
Weights sum:  19.866675395574447


outer iteration -  500
Accuracy  0.9897959183673469
Weights sum:  20.531486361423255


outer iteration -  600
Accuracy  0.9897959183673469
Weights sum:  21.078610295555322


outer iteration -  700
Accuracy  0.9897959183673469
Weights sum:  21.551432463523202


outer iteration -  800
Accuracy  0.9876093294460642
Weights sum:  21.94502680475646


outer iteration -  900
Accuracy  0.9876093294460642
Weights sum:  22.16942461523437




# 2. Wisconsin breast cancer

In [238]:
# https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/

data = pd.read_csv("data/breast-cancer-wisconsin.data", header = None).iloc[:,1:11]


data.head()

,1,2,3,4,5,6,7,8,9,10
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [239]:
X = data.iloc[:,0:9]
X.head()

Y = data.iloc[:, 9]

supermapping = {2: -1, 4: 1}

Y = Y.map(supermapping)

In [240]:
X.dtypes
X = X.drop(6, axis = 1)

In [58]:
X.dtypes

1    int64
2    int64
3    int64
4    int64
5    int64
7    int64
8    int64
9    int64
dtype: object

In [241]:
X = np.array(X)
Y = np. array(Y)

# free term
X = np.concatenate([X, np.ones(shape = (X.shape[0], 1))], axis=1)




In [183]:
X

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 2., 2., ..., 2., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.]])

In [184]:
for i in range(X.shape[1]):
  mean = X[:,i].mean()
  std = X[:,i].std()
  if std == 0: #If column of zeros
    std = 1
    print("Std is Zero")
    print(i)
  X[:,i] = (X[:,i] - mean)/std

Std is Zero
8


In [242]:
# svc from library
from sklearn.svm import LinearSVC
svc = LinearSVC(penalty = "l1", loss = "squared_hinge", max_iter = 100, C = 5, dual = False) # last 2 params - same as in our algorithm


svc.fit(X,Y)

svc.score(X,Y)

C:\Users\Patryk\anaconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9628040057224606

In [105]:
X = X[:,  :-3]

# free term
X = np.concatenate([X, np.ones(shape = (X.shape[0], 1))], axis=1)

In [143]:
X[:,2].std()

2.9697861746233842

In [131]:
X

array([[ 5.,  1.,  1., ...,  1.,  1.,  1.],
       [ 5.,  4.,  4., ...,  2.,  1.,  1.],
       [ 3.,  1.,  1., ...,  1.,  1.,  1.],
       ...,
       [ 5., 10., 10., ..., 10.,  2.,  1.],
       [ 4.,  8.,  6., ...,  6.,  1.,  1.],
       [ 4.,  8.,  8., ...,  4.,  1.,  1.]])

In [243]:
# algorithm described in https://jmlr.org/papers/volume9/chang08a/chang08a.pdf


n_of_features = X.shape[1]
n_of_observations = Y.shape[0]


# initial weights

#weights = np.ones(shape = (1,n_of_features) ) * 1 # initial weights of predictors
weights = np.ones((n_of_features)) * 100

# hyperparameters
beta = 0.5
sigma = 0.01
C = 5

# outer iterations:
for k in range(1000):
    # inner iterations shuld be as many as the number of features
    for i in range(n_of_features):
#         if(i%100==0):
#           print("feature " , i)
        shuffle = np.random.choice(n_of_observations, n_of_observations, replace = False) # shuffling recommended by authors
        col = X[:,i] #index2Col(X, i)
        
        # reorder randomly column of X and target (Y):
        col_ = col[shuffle]
        Y_ = Y[shuffle] #np.array(Y.iloc[shuffle])
        X_ = X[shuffle,:]
        
        # solving the sub-problem and update weight_i
        
        ### calculating Newton direction d
        
        D_prim_0 = 0
        
        sum_of_squares_of_column = 0
        
        #----- calculating D'  for z = 0 , in the article: equation (9)
        for j in range(n_of_observations): 
            #----- b_j defined in thr article, part of D' calculations
            b_j = max(Y_[j]*float(np.dot(weights,X_[j,:])),0)
            #print(float(np.dot(weights,index2Row(df,j))),b_j)
            if b_j > 0.0: #----- indicator defined in the article
                D_prim_0 += Y_[j]*col_[j]*b_j
                sum_of_squares_of_column += col_[j]**2
            
            
        D_prim_0 *= (-2*C)

        #D_prim_0 += weights[0,i]
        #D_prim_0 += weights[i]


        #----- calculating D''  for z = 0 , in the article: equation (10)
        D_bis_0 = 1 + 2*C*sum_of_squares_of_column

        #-----  equation (13) in the article
        d = -D_prim_0/D_bis_0 
        ### compute lambda = max{1, beta, beta^2, beta^3, ...}
        
        # we do this to ensure sufficient convergence condition & to make calculations quicker
        # Theorem 1 ( from article) indicated that lambda_ (14) fulfilling (15) 
        
        H_i = 1 + 2*C*np.dot(col_.T,col_)
        lambda_ = D_bis_0 /((H_i/2)+sigma)
        
        lambd = 1
        while lambd > lambda_:
            lambd *= beta
            
        # update weights
        weights[i] -= lambd*d
    if k % 100 == 0:
#     if True:
        print("outer iteration - ", k)
        acc = 0.0
        for i in range(n_of_observations):
          if np.dot(weights,X[i,:])*Y[i] < 0:
            acc += 1
        print("Accuracy ",acc/n_of_observations)
        print("Weights sum: ",weights.sum())
        print("\n")



outer iteration -  0
Accuracy  0.5464949928469242
Weights sum:  719.437755224928


outer iteration -  100
Accuracy  0.9427753934191703
Weights sum:  1201.9434639341212


outer iteration -  200
Accuracy  0.9399141630901288
Weights sum:  1037.2011213298356


outer iteration -  300
Accuracy  0.944206008583691
Weights sum:  880.8828587642118


outer iteration -  400
Accuracy  0.944206008583691
Weights sum:  749.053625787788


outer iteration -  500
Accuracy  0.9456366237482118
Weights sum:  637.6004987887269


outer iteration -  600
Accuracy  0.944206008583691
Weights sum:  543.8210590931088




KeyboardInterrupt: 

In [245]:
# algorithm described in https://jmlr.org/papers/volume9/chang08a/chang08a.pdf


n_of_features = X.shape[1]
n_of_observations = Y.shape[0]


# initial weights

#weights = np.ones(shape = (1,n_of_features) ) * 1 # initial weights of predictors
weights = np.ones((n_of_features)) * -100

# hyperparameters
beta = 0.5
sigma = 0.01
C = 5

# outer iterations:
for k in range(1000):
    # inner iterations shuld be as many as the number of features
    for i in range(n_of_features):
#         if(i%100==0):
#           print("feature " , i)
        shuffle = np.random.choice(n_of_observations, n_of_observations, replace = False) # shuffling recommended by authors
        col = X[:,i] #index2Col(X, i)
        
        # reorder randomly column of X and target (Y):
        col_ = col[shuffle]
        Y_ = Y[shuffle] #np.array(Y.iloc[shuffle])
        X_ = X[shuffle,:]
        
        # solving the sub-problem and update weight_i
        
        ### calculating Newton direction d
        
        D_prim_0 = 0
        
        sum_of_squares_of_column = 0
        
        #----- calculating D'  for z = 0 , in the article: equation (9)
        for j in range(n_of_observations): 
            #----- b_j defined in thr article, part of D' calculations
            b_j = max(Y_[j]*float(np.dot(weights,X_[j,:])),0)
            #print(float(np.dot(weights,index2Row(df,j))),b_j)
            if b_j > 0.0: #----- indicator defined in the article
                D_prim_0 += Y_[j]*col_[j]*b_j
                sum_of_squares_of_column += col_[j]**2
            
            
        D_prim_0 *= (-2*C)

        #D_prim_0 += weights[0,i]
        #D_prim_0 += weights[i]


        #----- calculating D''  for z = 0 , in the article: equation (10)
        D_bis_0 = 1 + 2*C*sum_of_squares_of_column

        #-----  equation (13) in the article
        d = -D_prim_0/D_bis_0 
        ### compute lambda = max{1, beta, beta^2, beta^3, ...}
        
        # we do this to ensure sufficient convergence condition & to make calculations quicker
        # Theorem 1 ( from article) indicated that lambda_ (14) fulfilling (15) 
        
        H_i = 1 + 2*C*np.dot(col_.T,col_)
        lambda_ = D_bis_0 /((H_i/2)+sigma)
        
        lambd = 1
        while lambd > lambda_:
            lambd *= beta
            
        # update weights
        weights[i] -= lambd*d
    if k % 100 == 0:
#     if True:
        print("outer iteration - ", k)
        acc = 0.0
        for i in range(n_of_observations):
          if np.dot(weights,X[i,:])*Y[i] < 0:
            acc += 1
        print("Accuracy ",acc/n_of_observations)
        print("Weights sum: ",weights.sum())
        print("\n")



outer iteration -  0
Accuracy  0.6738197424892703
Weights sum:  -15.533713864091453


outer iteration -  100
Accuracy  0.9413447782546495
Weights sum:  106.34219318891958


outer iteration -  200
Accuracy  0.949928469241774
Weights sum:  99.93456249351736


outer iteration -  300
Accuracy  0.9513590844062947
Weights sum:  87.76409865849202


outer iteration -  400
Accuracy  0.9513590844062947
Weights sum:  76.61881618127036


outer iteration -  500
Accuracy  0.9513590844062947
Weights sum:  67.37694967893914


outer iteration -  600
Accuracy  0.949928469241774
Weights sum:  59.10135826029666


outer iteration -  700
Accuracy  0.9484978540772532
Weights sum:  51.969878545492286


outer iteration -  800
Accuracy  0.9470672389127325
Weights sum:  45.933842785060115


outer iteration -  900
Accuracy  0.9484978540772532
Weights sum:  40.48627866059195




# 3. Haberman data

In [200]:
# https://archive.ics.uci.edu/ml/datasets/Haberman%27s+Survival

data = pd.read_csv("data/haberman.data", header= None)

In [209]:
X = data.iloc[:, 0:3]

Y = data.iloc[:,3]

supermapping = {1: -1, 2: 1}

Y = Y.map(supermapping)

In [210]:
np.unique(Y)

array([-1,  1], dtype=int64)

In [212]:
X.std()

0    10.803452
1     3.249405
2     7.189654
dtype: float64

In [213]:
X = np.array(X)
Y = np. array(Y)

In [214]:
for i in range(X.shape[1]):
  mean = X[:,i].mean()
  std = X[:,i].std()
  if std == 0: #If column of zeros
    std = 1
    print("Std is Zero")
    print(i)
  X[:,i] = (X[:,i] - mean)/std

In [215]:
# free term
X = np.concatenate([X, np.ones(shape = (X.shape[0], 1))], axis=1)

In [216]:
# svc from library
from sklearn.svm import LinearSVC
svc = LinearSVC(penalty = "l1", loss = "squared_hinge", max_iter = 100, C = 5, dual = False) # last 2 params - same as in our algorithm


svc.fit(X,Y)

svc.score(X,Y)

0.7320261437908496

In [237]:
# algorithm described in https://jmlr.org/papers/volume9/chang08a/chang08a.pdf


n_of_features = X.shape[1]
n_of_observations = Y.shape[0]


# initial weights

#weights = np.ones(shape = (1,n_of_features) ) * 1 # initial weights of predictors
weights = np.ones((n_of_features)) * 100

# hyperparameters
beta = 0.5
sigma = 0.01
C = 5

# outer iterations:
for k in range(3000):
    # inner iterations shuld be as many as the number of features
    for i in range(n_of_features):
#         if(i%100==0):
#           print("feature " , i)
        shuffle = np.random.choice(n_of_observations, n_of_observations, replace = False) # shuffling recommended by authors
        col = X[:,i] #index2Col(X, i)
        
        # reorder randomly column of X and target (Y):
        col_ = col[shuffle]
        Y_ = Y[shuffle] #np.array(Y.iloc[shuffle])
        X_ = X[shuffle,:]
        
        # solving the sub-problem and update weight_i
        
        ### calculating Newton direction d
        
        D_prim_0 = 0
        
        sum_of_squares_of_column = 0
        
        #----- calculating D'  for z = 0 , in the article: equation (9)
        for j in range(n_of_observations): 
            #----- b_j defined in thr article, part of D' calculations
            b_j = max(Y_[j]*float(np.dot(weights,X_[j,:])),0)
            #print(float(np.dot(weights,index2Row(df,j))),b_j)
            if b_j > 0.0: #----- indicator defined in the article
                D_prim_0 += Y_[j]*col_[j]*b_j
                sum_of_squares_of_column += col_[j]**2
            
            
        D_prim_0 *= (-2*C)

        #D_prim_0 += weights[0,i]
        #D_prim_0 += weights[i]


        #----- calculating D''  for z = 0 , in the article: equation (10)
        D_bis_0 = 1 + 2*C*sum_of_squares_of_column

        #-----  equation (13) in the article
        d = -D_prim_0/D_bis_0 
        ### compute lambda = max{1, beta, beta^2, beta^3, ...}
        
        # we do this to ensure sufficient convergence condition & to make calculations quicker
        # Theorem 1 ( from article) indicated that lambda_ (14) fulfilling (15) 
        
        H_i = 1 + 2*C*np.dot(col_.T,col_)
        lambda_ = D_bis_0 /((H_i/2)+sigma)
        
        lambd = 1
        while lambd > lambda_:
            lambd *= beta
            
        # update weights
        weights[i] -= lambd*d*0.1
    if k % 100 == 0:
#     if True:
        print("outer iteration - ", k)
        acc = 0.0
        for i in range(n_of_observations):
          if np.dot(weights,X[i,:])*Y[i] < 0:
            acc += 1
        print("Accuracy ",acc/n_of_observations)
        print("Weights sum: ",weights.sum())
        print("\n")



outer iteration -  0
Accuracy  0.5915032679738562
Weights sum:  373.2073070163555


outer iteration -  100
Accuracy  0.7124183006535948
Weights sum:  7.2850370191001765


outer iteration -  200
Accuracy  0.7418300653594772
Weights sum:  0.012721800357905766


outer iteration -  300
Accuracy  0.738562091503268
Weights sum:  -0.007007570996867017


outer iteration -  400
Accuracy  0.738562091503268
Weights sum:  -0.0006041915866221831


outer iteration -  500
Accuracy  0.738562091503268
Weights sum:  -4.9744453494224435e-05


outer iteration -  600
Accuracy  0.738562091503268
Weights sum:  -4.0711608376389045e-06




KeyboardInterrupt: 

# Full loaded data

In [ ]:
X = np.array(X)
Y = np.array(target_df)

In [ ]:
X.shape

(72309, 20958)

In [ ]:
for i in range(20958-1):
  mean = X[:,i].mean()
  std = X[:,i].std()
  if std == 0: #If column of zeros
    std = 1
    print("Std is Zero, ID:", i)
    
  X[:,i] = (X[:,i] - mean)/std

Std is Zero
0
Std is Zero
9646
Std is Zero
10252
Std is Zero
10597
Std is Zero
10618
Std is Zero
11565
Std is Zero
12211
Std is Zero
12241
Std is Zero
12263
Std is Zero
12279
Std is Zero
12948
Std is Zero
14052
Std is Zero
14617
Std is Zero
15313


IndexError: ignored

In [ ]:
# algorithm described in https://jmlr.org/papers/volume9/chang08a/chang08a.pdf


n_of_features = X.shape[1]
n_of_observations = Y.shape[0]


# initial weights

#weights = np.ones(shape = (1,n_of_features) ) * 0.01 # initial weights of predictors
weights = np.ones((n_of_features)) * 0.01

# hyperparameters
beta = 0.5
sigma = 0.01
C = 0.01

# outer iterations:
for k in range(4):
    # inner iterations shuld be as many as the number of features
    for i in range(n_of_features):
        startTime = time.time()
      
        if(i%100==0):
          print("feature " , i)
        shuffle = np.random.choice(n_of_observations, n_of_observations, replace = False) # shuffling recommended by authors
        col = X[:,i] #index2Col(X, i)
        
        # reorder randomly column of X and target (Y):
        col_ = col[shuffle]
        Y_ = Y[shuffle] #np.array(Y.iloc[shuffle])
        
        
        # solving the sub-problem and update weight_i
        
        ### calculating Newton direction d
        
        D_prim_0 = 0
        
        sum_of_squares_of_column = 0
        
        for j in range(n_of_observations): 

            b_j = max(Y_[j]*float(np.dot(weights,X[j,:])),0)
            #print(float(np.dot(weights,index2Row(df,j))),b_j)
            if b_j > 0.0:
                D_prim_0 += Y_[j]*col_[j]*b_j
                sum_of_squares_of_column += col_[j]**2
            
            
        D_prim_0 *= (-2*C)
        #D_prim_0 += weights[0,i]
        D_prim_0 += weights[i]
                
        D_bis_0 = 1 + 2*C*sum_of_squares_of_column
        d = -D_prim_0/D_bis_0
        
        ### compute lambda = max{1, beta, beta^2, beta^3, ...}
        
        # we do this to ensure sufficient convergence condition & to make calculations quicker
        
        H_i = 1 + 2*C*np.dot(col_.T,col_)
        lambda_ = D_bis_0 /((H_i/2)+sigma)
        
        lambd = 1
        while lambd > lambda_:
            lambd *= beta
            
        # update weights
        weights[i] += lambd*d
        executionTime = (time.time() - startTime)
        print('Execution time in seconds: ' + str(executionTime))
    print("outer iteration - ", k)
    acc = 0.0
    for i in range(d_d):
      if np.dot(weights,X[i,:])*Y[i] > 0:
        acc += 1
    print(acc/d_d)





feature  0
Execution time in seconds: 14.164132833480835
Execution time in seconds: 10.2814302444458
Execution time in seconds: 10.202035665512085
Execution time in seconds: 10.176557779312134
Execution time in seconds: 10.259861707687378


KeyboardInterrupt: ignored